In [ ]:
!pip install tensorflow tensorflow-text datasets nltk rouge pydot graphviz
import nltk
nltk.download('punkt')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: m

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Concatenate, Dropout, Bidirectional, MultiHeadAttention
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import os
import pickle
from google.colab import drive
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from nltk.translate import bleu_score

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')
model_path = '/content/drive/My Drive/model_BiLSTM_word_reinforcement_V2.h5'
history_path = '/content/drive/My Drive/history_BiLSTM_word_reinforcement_V2.pkl'

Mounted at /content/drive


In [ ]:
# Load dataset
dataset = load_dataset("neulab/tldr", split='train')

# Append <start> and <end> tokens to cmd_texts
cmd_texts = ["<start> " + item['cmd'] + " <end>" for item in dataset]

# Combine nl_texts and cmd_texts for tokenizer fitting
nl_texts = [item['nl'] for item in dataset]
all_texts = nl_texts + cmd_texts

# Initialize and fit tokenizer
tokenizer = Tokenizer(filters='', lower=True, split=' ')
tokenizer.fit_on_texts(all_texts)
vocab_size = len(tokenizer.word_index) + 1  # +1 for zero padding

# Prepare sequences
nl_sequences = tokenizer.texts_to_sequences(nl_texts)
cmd_sequences = tokenizer.texts_to_sequences(cmd_texts)

# Determine maximum sequence length
max_length = max(max(len(seq) for seq in nl_sequences), max(len(seq) for seq in cmd_sequences))

# Pad sequences
nl_sequences_padded = pad_sequences(nl_sequences, maxlen=max_length, padding='post')
cmd_sequences_padded = pad_sequences(cmd_sequences, maxlen=max_length, padding='post')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for neulab/tldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/neulab/tldr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
# Prepare decoder input data and target data
decoder_input_data = np.zeros_like(cmd_sequences_padded)
decoder_input_data[:, 1:] = cmd_sequences_padded[:, :-1]

# RL Training Setup
optimizer = Adam(learning_rate=1e-3)
epochs = 200
batch_size = 64
def build_model(vocab_size, max_length):
    # Define the dimensionality of the embedding and LSTM units
    embedding_dim = 256
    lstm_units = 256

    # Encoder
    encoder_inputs = Input(shape=(None,), name='encoder_inputs')
    encoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='encoder_embedding')(encoder_inputs)
    encoder_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True, name='encoder_lstm'), name='bidirectional_encoder')
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)

    # Since we are using a bidirectional LSTM, we need to concatenate the final states
    state_h = Concatenate(name='encoder_state_h')([forward_h, backward_h])
    state_c = Concatenate(name='encoder_state_c')([forward_c, backward_c])

    # Decoder
    decoder_inputs = Input(shape=(None,), name='decoder_inputs')
    decoder_embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, name='decoder_embedding')(decoder_inputs)
    decoder_lstm = LSTM(2 * lstm_units, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

    # Attention
    attention_layer = MultiHeadAttention(num_heads=4, key_dim=2 * lstm_units, name='multihead_attention')
    attention_output = attention_layer(query=decoder_outputs, key=encoder_outputs, value=encoder_outputs)

    # Concatenate attention input and decoder LSTM output
    decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attention_output])

    # Dense layer to predict each word probability
    dense_output = TimeDistributed(Dense(vocab_size, activation='softmax'), name='output_dense')(decoder_concat_input)

    # Define the model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=dense_output)

    # Compile the model with a suitable optimizer, loss, and metric
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()  # Show model architecture

    return model

# Instantiate and compile the model
model = build_model(vocab_size, max_length)
optimizer = Adam(learning_rate=1e-3)
patience = 10  # Define patience for early stopping
best_bleu = 0   # Initialize the best BLEU score tracker




Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_embedding         │ (None, None, 256)      │      3,058,688 │ encoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_encoder     │ [(None, None, 512),    │      1,050,624 │ encoder_embedding[0][… │
│ (Bidirectional)           │ (None, 256), (None,    │                │                        │
│                           │ 256), (None, 256),     │                │                        │
│                           │ (None, 256)]           │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_embedding         │ (None, None, 256)      │      3,058,688 │ decoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_state_h           │ (None, 512)            │              0 │ bidirectional_encoder… │
│ (Concatenate)             │                        │                │ bidirectional_encoder… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_state_c           │ (None, 512)            │              0 │ bidirectional_encoder… │
│ (Concatenate)             │                        │                │ bidirectional_encoder… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ [(None, None, 512),    │      1,574,912 │ decoder_embedding[0][… │
│                           │ (None, 512), (None,    │                │ encoder_state_h[0][0], │
│                           │ 512)]                  │                │ encoder_state_c[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multihead_attention       │ (None, None, 512)      │      4,200,960 │ bidirectional_encoder… │
│ (MultiHeadAttention)      │                        │                │ decoder_lstm[0][0],    │
│                           │                        │                │ bidirectional_encoder… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concat_layer              │ (None, None, 1024)     │              0 │ decoder_lstm[0][0],    │
│ (Concatenate)             │                        │                │ multihead_attention[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output_dense              │ (None, None, 11948)    │     12,246,700 │ concat_layer[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 25,190,572 (96.09 MB)

 Trainable params: 25,190,572 (96.09 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from nltk.translate import bleu_score

from nltk.translate.bleu_score import SmoothingFunction

def compute_bleu(references, predictions):
    """ Compute BLEU score for batch predictions with smoothing """
    smoothie = SmoothingFunction().method1  # Using smoothing technique method1
    bleu_scores = [sentence_bleu([ref.split()], pred.split(), weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie) for ref, pred in zip(references, predictions)]
    return np.mean(bleu_scores)


def rl_training_step(input_seqs, target_seqs, model, tokenizer, optimizer):
    """ Perform a training step and return reward and loss """
    with tf.GradientTape() as tape:
        predictions = model([input_seqs, np.zeros_like(input_seqs)], training=True)
        loss = SparseCategoricalCrossentropy(from_logits=False)(target_seqs, predictions)

        # Convert predictions to text
        predicted_sequences = tf.argmax(predictions, axis=-1)
        predicted_texts = tokenizer.sequences_to_texts(predicted_sequences.numpy())
        target_texts = tokenizer.sequences_to_texts(target_seqs)

        # Compute BLEU score
        rewards = compute_bleu(target_texts, predicted_texts)
        reward = tf.reduce_mean(rewards)
        reward = tf.cast(reward, dtype=tf.float32)

        # Compute combined loss
        combined_loss = -tf.math.log(reward + 1e-9) + loss

    gradients = tape.gradient(combined_loss, model.trainable_variables)
    if gradients is None or any(g is None for g in gradients):
        print("No valid gradients. Check model and loss configuration.")
        return 0, 0

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return reward.numpy(), loss.numpy()


# Example use in the training loop
for epoch in range(epochs):
    indices = np.random.permutation(len(nl_sequences_padded))
    epoch_rewards = []
    epoch_losses = []
    for i in range(0, len(nl_sequences_padded), batch_size):
        batch_indices = indices[i:i + batch_size]
        reward, loss = rl_training_step(
            nl_sequences_padded[batch_indices],
            cmd_sequences_padded[batch_indices],
            model,
            tokenizer,
            optimizer
        )
        epoch_rewards.append(reward)
        epoch_losses.append(loss)

    avg_reward = np.mean(epoch_rewards)
    avg_loss = np.mean(epoch_losses)
    print(f"Epoch {epoch+1}: Average BLEU Reward = {avg_reward}, Loss = {avg_loss}")
    if avg_reward > best_bleu:
        best_bleu = avg_reward
        model.save(model_path)
        print(f"New best model saved with BLEU score: {best_bleu}")
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping...")
            break


Epoch 1: Average BLEU Reward = 0.0552549734711647, Loss = 1.427756428718567
New best model saved with BLEU score: 0.0552549734711647


Epoch 2: Average BLEU Reward = 0.06440562754869461, Loss = 0.9528326392173767
New best model saved with BLEU score: 0.06440562754869461


Epoch 3: Average BLEU Reward = 0.06533413380384445, Loss = 0.8930621147155762
New best model saved with BLEU score: 0.06533413380384445


Epoch 4: Average BLEU Reward = 0.06564897298812866, Loss = 0.8492655754089355
New best model saved with BLEU score: 0.06564897298812866


Epoch 5: Average BLEU Reward = 0.06650618463754654, Loss = 0.8080022931098938
New best model saved with BLEU score: 0.06650618463754654


...

Epoch 71: Average BLEU Reward = 0.904762327671051, Loss = 0.030205441638827324
New best model saved with BLEU score: 0.904762327671051


Epoch 72: Average BLEU Reward = 0.9104154109954834, Loss = 0.02951730042695999
New best model saved with BLEU score: 0.9104154109954834
Epoch 73: Average BLEU Reward = 0.9102648496627808, Loss = 0.03051701933145523


Epoch 74: Average BLEU Reward = 0.9109330773353577, Loss = 0.02981886826455593
New best model saved with BLEU score: 0.9109330773353577


Exception ignored in: <function _xla_gc_callback at 0x7b61d0eb0ee0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# Load the model
model_path = '/content/drive/My Drive/model_BiLSTM_word_reinforcement_V2.h5'
model = load_model(model_path)
def translate(model, tokenizer, text, max_length):
    # Tokenize the input text
    sequence = tokenizer.texts_to_sequences([text])
    sequence_padded = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Prepare the decoder input data
    decoder_input_data = np.zeros_like(sequence_padded)
    decoder_input_data[:, 1:] = sequence_padded[:, :-1]

    # Predict
    prediction = model.predict([sequence_padded, decoder_input_data])

    # Convert prediction to text
    predicted_sequence = np.argmax(prediction[0], axis=-1)
    predicted_text = tokenizer.sequences_to_texts([predicted_sequence])

    # Remove the <start> and <end> tokens
    predicted_text_clean = [token for token in predicted_text[0].split() if token not in ['<start>', '<end>']]

    # Join the tokens back together
    predicted_command = ' '.join(predicted_text_clean)

    return predicted_command

In [ ]:
# Get the first 10 rows of the training dataset
first_10_nl_texts = nl_texts[:100]
first_10_cmd_texts = cmd_texts[:100]

# Translate each text and print the input, prediction, and actual command
for i in range(100):
    input_text = first_10_nl_texts[i]
    actual_command = first_10_cmd_texts[i]
    predicted_command = translate(model, tokenizer, input_text, max_length)

    print(f"Input Text: {input_text}")
    print(f"Predicted Command: {predicted_command}")
    print(f"Actual Command: {actual_command}")
    print("\n---\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Input Text: <start> get the label of a fat32 partition <end>
Predicted Command: decode members generate unused logs
Actual Command: <start> fatlabel {{/dev/sda1}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Input Text: <start> set the label of a fat32 partition <end>
Predicted Command: ascii framework generate {{file.pdf}} logs {{foo}}
Actual Command: <start> fatlabel {{/dev/sdc3}} "{{new_label}}" <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Input Text: <start> search for a package in your current sources <end>
Predicted Command: "{{title}}" docker file,
Actual Command: <start> apt-cache search {{query}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
Input Text: <start> show information about a package <end>
Predicted Command: sudo default
Actual Command: <start> apt-cache show {{package}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Input Text: <start> show whether a package is installed and up to date <end>
Predicted Command: 

In [ ]:
# Load dataset
dataset_val = load_dataset("neulab/tldr", split='validation')

# Append <start> and <end> tokens to cmd_texts
cmd_text_val = ["<start> " + item['cmd'] + " <end>" for item in dataset_val]

# Combine nl_texts and cmd_texts for tokenizer fitting
nl_texts_val = [item['nl'] for item in dataset_val]

# Get the first 10 rows of the training dataset
first_100_nl_texts_val = nl_texts[:100]
first_100_cmd_texts_val = cmd_texts[:100]

# Translate each text and print the input, prediction, and actual command
for i in range(100):
    input_text = first_100_nl_texts_val[i]
    actual_command = first_100_cmd_texts_val[i]
    predicted_command = translate(model, tokenizer, input_text, max_length)

    print(f"Input Text: {input_text}")
    print(f"Predicted Command: {predicted_command}")
    print(f"Actual Command: {actual_command}")
    print("\n---\n")



/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for neulab/tldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/neulab/tldr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Input Text: <start> get the label of a fat32 partition <end>
Predicted Command: decode members generate unused logs
Actual Command: <start> fatlabel {{/dev/sda1}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Input Text: <start> set the label of a fat32 partition <end>
Predicted Command: ascii framework generate {{file.pdf}} logs {{foo}}
Actual Command: <start> fatlabel {{/dev/sdc3}} "{{new_label}}" <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Input Text: <start> search for a package in your current sources <end>
Predicted Command: "{{title}}" docker file,
Actual Command: <start> apt-cache search {{query}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Input Text: <start> show information about a package <end>
Predicted Command: sudo default
Actual Command: <start> apt-cache show {{package}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Input Text: <start> show whether a package is installed and up to date <end>
Predicted Command: s

In [ ]:
# Load dataset
dataset_test = load_dataset("neulab/tldr", split='test')

# Append <start> and <end> tokens to cmd_texts
cmd_text_test = ["<start> " + item['cmd'] + " <end>" for item in dataset_val]

# Combine nl_texts and cmd_texts for tokenizer fitting
nl_texts_test = [item['nl'] for item in dataset_val]

# Get the first 10 rows of the training dataset
first_100_nl_texts_test = nl_texts[:100]
first_100_cmd_texts_test = cmd_texts[:100]

# Translate each text and print the input, prediction, and actual command
for i in range(100):
    input_text = first_100_nl_texts_test[i]
    actual_command = first_100_cmd_texts_test[i]
    predicted_command = translate(model, tokenizer, input_text, max_length)

    print(f"Input Text: {input_text}")
    print(f"Predicted Command: {predicted_command}")
    print(f"Actual Command: {actual_command}")
    print("\n---\n")



/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for neulab/tldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/neulab/tldr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Input Text: <start> get the label of a fat32 partition <end>
Predicted Command: decode members generate unused logs
Actual Command: <start> fatlabel {{/dev/sda1}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Input Text: <start> set the label of a fat32 partition <end>
Predicted Command: ascii framework generate {{file.pdf}} logs {{foo}}
Actual Command: <start> fatlabel {{/dev/sdc3}} "{{new_label}}" <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Input Text: <start> search for a package in your current sources <end>
Predicted Command: "{{title}}" docker file,
Actual Command: <start> apt-cache search {{query}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Input Text: <start> show information about a package <end>
Predicted Command: sudo default
Actual Command: <start> apt-cache show {{package}} <end>

---

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Input Text: <start> show whether a package is installed and up to date <end>
Predicted Command: s